In [84]:
import numpy as np
from sklearn.datasets import make_classification, load_iris
from sklearn.model_selection import train_test_split
import torch

In [85]:
# X, y = make_classification(n_samples=250, n_informative = 7, n_classes=3, n_features=10)
X,y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [86]:
from simple_model import ClassifierModel, DyadRankingModel

clf = ClassifierModel(input_dim = X_train.shape[1], hidden_dim=16, output_dim=y.max()+1)
rnk = DyadRankingModel(input_dim = X_train.shape[1] + y.max()+1, hidden_dim=16)

clf.fit(X_train,y_train, num_epochs=100, batch_size=1, learning_rate=0.001)
rnk.fit(X_train,y_train, num_epochs=100, batch_size=1, learning_rate=0.001)

In [87]:
from sklearn.metrics import accuracy_score
y_pred_clf = clf.predict(X_test)
y_pred_rnk = rnk.predict_classes(X_test)


In [88]:
accuracy_score(y_pred_clf, y_test)

0.84

In [89]:
accuracy_score(y_pred_rnk, y_test)

0.32

In [77]:
print(y_pred_rnk)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [30]:
X = torch.tensor(X_test, dtype=torch.float32)
eye = torch.eye(3)
eyes = eye.repeat((X.shape[0], 1))
Xs = X.repeat_interleave(3, dim=0)
dyads = torch.cat([Xs, eyes], dim=1)

In [31]:
rnk(dyads).view(-1,3)

tensor([[-0.4796, -0.5233, -0.4966],
        [-0.5095, -0.5378, -0.5236],
        [-0.5111, -0.5353, -0.5221],
        [-0.4857, -0.5195, -0.5007],
        [-0.5175, -0.5392, -0.5261],
        [-0.4947, -0.5275, -0.5089],
        [-0.4765, -0.5204, -0.4935],
        [-0.5084, -0.5380, -0.5205],
        [-0.4841, -0.5184, -0.4992],
        [-0.4742, -0.5204, -0.4936],
        [-0.4633, -0.5121, -0.4851],
        [-0.5059, -0.5355, -0.5177],
        [-0.4742, -0.5183, -0.4922],
        [-0.5144, -0.5391, -0.5254],
        [-0.4878, -0.5211, -0.5023],
        [-0.4908, -0.5231, -0.5042],
        [-0.4775, -0.5221, -0.4951],
        [-0.4853, -0.5276, -0.5008],
        [-0.4910, -0.5320, -0.5047],
        [-0.4935, -0.5261, -0.5078],
        [-0.4989, -0.5292, -0.5112],
        [-0.4917, -0.5238, -0.5059],
        [-0.5081, -0.5387, -0.5216],
        [-0.4757, -0.5121, -0.4930],
        [-0.4981, -0.5273, -0.5101],
        [-0.5070, -0.5319, -0.5180],
        [-0.4689, -0.5149, -0.4883],
 